# アウトバウンド認証

アウトバウンド認証により、エージェントと AgentCore Gateway は、インバウンド認証で認証および認可されたユーザーに代わって、AWS リソースとサードパーティサービスに安全にアクセスできます。AWS リソースまたはサードパーティサービスとの認可を統合するには、インバウンド認証とアウトバウンド認証の両方を設定する必要があります。

AgentCore Identity がサポートするジャストイナフアクセスと安全な権限委任により、エージェントは GitHub、Google、Salesforce、Slack などの AWS リソースとサードパーティツールにシームレスかつ安全にアクセスできます。エージェントは、事前に認可されたユーザーの同意があれば、ユーザーに代わって、または独立してこれらのサービスでアクションを実行できます。さらに、安全なトークンボールトを使用して同意疲れを軽減し、合理化された AI エージェントエクスペリエンスを作成できます。

## アウトバウンド認証の設定

まず、クライアントアプリケーションをサードパーティプロバイダーに登録し、アウトバウンド認証を作成します。AWS リソース、サードパーティサービス、または AgentCore Gateway ターゲットへのアクセスを検証する方法を指定します。OAuth 2LO/3LO または API キーを使用できます。OAuth では、AgentCore Identity が提供するプロバイダーから選択できます。その場合、AgentCore Identity からプロバイダーの設定詳細を入力します。または、カスタムプロバイダーの詳細を提供することもできます。

ユーザーが AWS リソース、サードパーティサービス、または AgentCore Gateway ターゲットへのアクセスを希望する場合、アウトバウンド認証はインカミング認証によって提供されたアクセストークンが有効であることを確認し、有効であればリソースへのアクセスを許可します。

<div style="text-align:center">
    <img src="images/outbound_auth.png" width="90%"/>
</div>

## リソース資格情報プロバイダー

これは、エージェントコードがダウンストリームリソースサーバー（例：Google、GitHub）の資格情報を取得してアクセスするために使用するコンポーネントです。例えば、Gmail からメールを取得したり、Google カレンダーに会議を追加したりします。これにより、エージェント開発者がエンドユーザー、エージェントコード、外部認可サーバー間で 2LO および 3LO OAuth2 オーケストレーションフローを実装する負担が軽減されます。AgentCore は、カスタム OAuth2 資格情報プロバイダーと、認可サーバーエンドポイントとプロバイダー固有のパラメータが事前に設定された Google、GitHub、Slack、Salesforce などの組み込みプロバイダーリストの両方を提供します。

Bedrock AgentCore Identity は、OAuth2 または API キーをサポートする外部リソースでエージェント開発者が認証するための OAuth2 および API キー資格情報プロバイダーを提供します。次の例では、API キー資格情報プロバイダーの設定について説明します。エージェントは API キー資格情報プロバイダーを使用して、任意のエージェント操作用の API キーを取得できます。他の資格情報プロバイダーについてはドキュメントを参照してください。

### リソース資格情報プロバイダーの作成

以下は API キーリソース資格情報プロバイダーを作成する例です。

```
from bedrock_agentcore.services.identity import IdentityClient
identity_client = IdentityClient(region="us-west-2")

api_key_provider = identity_client.create_api_key_credential_provider({
    "name": "APIKey-provider-name",
    "apiKey": "<my-api-key>" # 外部アプリケーションベンダー（例：OpenAI）から取得した API キーに置き換えてください
})
print(api_key_provider)
```

### リソース資格情報プロバイダーからのアクセストークンまたは API キーの取得

以下は API キー資格情報プロバイダーから API キーを取得する例です。エージェントは API キーを使用して、LLM や API キー設定を使用する他のサービスと対話できます。資格情報プロバイダーから access_token や API キーなどの資格情報を取得するには、以下のように関数をデコレートできます。

```
import asyncio
from bedrock_agentcore.identity.auth import requires_access_token, requires_api_key

@requires_api_key(
    provider_name="APIKey-provider" # 独自の資格情報プロバイダー名に置き換えてください
)
async def need_api_key(*, api_key: str):
    print(f'async 関数で API キーを受信: {api_key}')

await need_api_key(api_key="")
```

以下は @require_access_token デコレータで使用できる各種パラメータです。


| パラメータ名         | 説明                                                                     |
|:--------------------|:------------------------------------------------------------------------|
| provider_name       | 資格情報プロバイダー名                                                    |
| into                | トークンを注入するパラメータ名                                            |
| scopes              | 要求する OAuth2 スコープ                                                  |
| on_auth_url         | 認可 URL を処理するコールバック                                           |
| auth_flow           | 認証フロータイプ（"M2M" または "USER_FEDERATION"）                        |
| callback_url        | OAuth2 コールバック URL                                                   |
| force_authentication| 再認証を強制                                                             |
| token_poller        | カスタムトークンポーラー実装                                              |



# Amazon Bedrock AgentCore Runtime で OpenAI モデルを使用した Strands Agents のホスティング

## 概要

このチュートリアルでは、01-AgentCore-runtime でデプロイした OpenAI モデルを使用するエージェントを変更し、API キー資格情報プロバイダーを使用するアウトバウンド認証用に設定します。OpenAI キーを保存する API キー資格情報プロバイダーをセットアップし、このキーを使用するようにエージェントコードを変更します。

### チュートリアルアーキテクチャ

<div style="text-align:center">
    <img src="images/outbound_auth_api.png" width="90%"/>
</div>


### チュートリアル詳細

| 情報                | 詳細                                                                     |
|:--------------------|:------------------------------------------------------------------------|
| チュートリアルタイプ | 対話型                                                                   |
| エージェントタイプ   | シングル                                                                 |
| エージェントフレームワーク | Strands Agents                                                     |
| LLMモデル           | GPT 4.1 mini                                                             |
| チュートリアルコンポーネント | AgentCore Runtime でのエージェントホスティング。Strands Agent と OpenAI Model の使用 |
| チュートリアル分野   | クロスバーティカル                                                       |
| 難易度              | 簡単                                                                     |
| 使用SDK             | Amazon BedrockAgentCore Python SDK および boto3                           |
| 資格情報プロバイダー | タイプ：API キー                                                          |


### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime でのエージェントホスティング
* OpenAI モデルの使用
* Strands Agents の使用
* API キー資格情報プロバイダーを使用した AgentCore エグレス認証の使用

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* AWS 資格情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker が実行中であること
* OpenAI API キー

OpenAI API キーの取得方法：
- OpenAI：[OpenAI API キー](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key)
- Azure OpenAI：[Azure OpenAI リソースを作成してキーを取得](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?tabs=azure-portal)

実行前に環境変数を設定：
- OpenAI：
  - `OPENAI_API_KEY`
- Azure OpenAI：
  - `AZURE_OPENAI_API_KEY`
  - `AZURE_OPENAI_ENDPOINT`
  - `AZURE_OPENAI_API_VERSION`（例：`2024-02-15-preview`）
  - `AZURE_OPENAI_DEPLOYMENT`（デプロイされたモデル名）

オプションのプロバイダー切り替え：
- `OPENAI_PROVIDER` = `openai`（デフォルト）または `azure`

注意：
- シークレットをハードコードしないでください。AgentCore Identity の資格情報プロバイダーを使用して、実行時に API キーを保存および取得し、定期的にキーをローテーションしてください。

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## エージェントの作成とローカルでの実験

エージェントを AgentCore Runtime にデプロイする前に、実験目的でローカルで開発および実行しましょう。

本番エージェントアプリケーションでは、エージェント作成プロセスとエージェント呼び出しを分離する必要があります。AgentCore Runtime では、エージェントの呼び出し部分を `@app.entrypoint` デコレータでデコレートし、ランタイムのエントリポイントとして設定します。まず、実験段階で各エージェントがどのように開発されるかを見てみましょう。

ここでのアーキテクチャは以下のようになります：

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [ ]:
%%writefile strands_agents_openai.py
from strands import Agent, tool
from strands_tools import calculator # calculator ツールをインポート
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os

## 以下の設定を Azure API キーの詳細で更新してください。
os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

# カスタムツールを作成
@tool
def weather():
    """ 天気を取得 """ # ダミー実装
    return "sunny"

model = "azure/gpt-4.1-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
)


agent = Agent(
    model=litellm_model,
    tools=[calculator, weather],
    system_prompt="あなたは親切なアシスタントです。簡単な数学計算と天気を教えることができます。"
)

def strands_agent_open_ai(payload):
    """
    ペイロードでエージェントを呼び出す
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_open_ai(json.loads(args.payload))
    print(response)

#### ローカルエージェントの呼び出し

In [ ]:
!python strands_agents_openai.py '{"prompt": "今の天気は何ですか？"}'

## リソース資格情報プロバイダーの作成

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient

from boto3.session import Session
import boto3
boto_session = Session()
region = boto_session.region_name

# API キープロバイダーの設定
identity_client = IdentityClient(region=region)

api_key_provider = identity_client.create_api_key_credential_provider({
    "name": "openai-apikey-provider",
    "apiKey": "<YOUR_API_KEY>" # 外部アプリケーションベンダー（例：OpenAI）から取得した API キーに置き換えてください
})
print(api_key_provider)


## AgentCore Runtime へのデプロイ用にエージェントを準備し、リソース資格情報プロバイダーを使用する

それでは、エージェントを AgentCore Runtime にデプロイしましょう。そのためには：
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp` で Runtime App をインポート
* `app = BedrockAgentCoreApp()` でコード内で App を初期化
* `@app.entrypoint` デコレータで呼び出し関数をデコレート
* `app.run()` で AgentCoreRuntime にエージェントの実行を制御させる
* 前のステップで作成したリソース資格情報プロバイダーから OpenAI キーを取得

### OpenAI モデルを使用した Strands Agents
GPT 4.1 mini モデルを使用する Strands Agent から始めましょう。他のモデルもまったく同じように動作します。

In [ ]:
%%writefile strands_agents_openai.py
import asyncio
from bedrock_agentcore.identity.auth import requires_access_token, requires_api_key
from strands import Agent, tool
from strands_tools import calculator 
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os
from bedrock_agentcore.runtime import BedrockAgentCoreApp

AZURE_API_KEY_FROM_CREDS_PROVIDER = ""


@requires_api_key(
    provider_name="openai-apikey-provider" # 独自の資格情報プロバイダー名に置き換えてください
)
async def need_api_key(*, api_key: str):
    global AZURE_API_KEY_FROM_CREDS_PROVIDER
    print(f'async 関数で API キーを受信: {api_key}')
    AZURE_API_KEY_FROM_CREDS_PROVIDER = api_key

# モジュールレベルで空の値を出力しない - エントリポイント関数で出力

app = BedrockAgentCoreApp()

# API キーはエントリポイント関数で動的に設定される
# 以下の設定を Azure API キーの詳細で更新してください。
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

# カスタムツールを作成
@tool
def weather():
    """ 天気を取得 """ # ダミー実装
    return "sunny"

# グローバルエージェント変数
agent = None

@app.entrypoint
async def strands_agent_open_ai(payload):
    """
    ペイロードでエージェントを呼び出す
    """
    global AZURE_API_KEY_FROM_CREDS_PROVIDER, agent
    
    print(f"エントリポイントが AZURE_API_KEY_FROM_CREDS_PROVIDER で呼び出されました: '{AZURE_API_KEY_FROM_CREDS_PROVIDER}'")
    
    # まだ取得していない場合は API キーを取得
    if not AZURE_API_KEY_FROM_CREDS_PROVIDER:
        print("API キーの取得を試行中...")
        try:
            await need_api_key(api_key="")
            print(f"API キーを取得しました: '{AZURE_API_KEY_FROM_CREDS_PROVIDER}'")
            os.environ["AZURE_API_KEY"] = AZURE_API_KEY_FROM_CREDS_PROVIDER
            print("環境変数 AZURE_API_KEY を設定しました")
        except Exception as e:
            print(f"API キー取得エラー: {e}")
            raise
    else:
        print("API キーは既に利用可能です")
    
    # API キーが設定された後にエージェントを初期化
    if agent is None:
        print("API キーでエージェントを初期化中...")
        model = "azure/gpt-4.1-mini"
        litellm_model = LiteLLMModel(
            model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
        )
        
        agent = Agent(
            model=litellm_model,
            tools=[calculator, weather],
            system_prompt="あなたは親切なアシスタントです。簡単な数学計算と天気を教えることができます。"
        )
        print("エージェントの初期化が成功しました")
    
    user_input = payload.get("prompt")
    print(f"ユーザー入力: {user_input}")
    
    try:
        response = agent(user_input)
        print(f"エージェント応答: {response}")
        return response.message['content'][0]['text']
    except Exception as e:
        print(f"エージェント処理エラー: {e}")
        raise

if __name__ == "__main__":
    app.run()

## 舞台裏で何が起こるか？

`BedrockAgentCoreApp` を使用すると、自動的に：

* ポート 8080 でリッスンする HTTP サーバーを作成
* エージェントの要件を処理するために必要な `/invocations` エンドポイントを実装
* ヘルスチェック用の `/ping` エンドポイントを実装（非同期エージェントにとって非常に重要）
* 適切なコンテンツタイプとレスポンス形式を処理
* AWS 標準に従ったエラー処理を管理

## エージェントを AgentCore Runtime にデプロイする

`CreateAgentRuntime` オペレーションは包括的な設定オプションをサポートしており、コンテナイメージ、環境変数、暗号化設定を指定できます。また、クライアントがエージェントと通信する方法を制御するためのプロトコル設定（HTTP、MCP）と認可メカニズムも設定できます。

**注意：** 運用のベストプラクティスは、コードをコンテナとしてパッケージ化し、CI/CD パイプラインと IaC を使用して ECR にプッシュすることです

このチュートリアルでは、Amazon Bedrock AgentCode Python SDK を使用してアーティファクトを簡単にパッケージ化し、AgentCore Runtime にデプロイします。

### AgentCore Runtime デプロイメントの設定

次に、スターターツールキットを使用して、エントリポイント、作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて Dockerfile が生成されます

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import boto3
import json
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()
agent_name="strands_agents_openai"

response = agentcore_runtime.configure(
    entrypoint="strands_agents_openai.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    agent_name=agent_name,
    requirements_file="requirements.txt",
    region=region
)
response

### AgentCore Runtime へのエージェントの起動

Dockerfile ができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

#### 自動作成されたロールに追加の必要なポリシーを追加

エージェントにアウトバウンド ID を追加しているため、自動作成されたロールでは利用できない API キーとシークレットを取得する必要があります。そのためには、自動作成された IAM ロールに追加の権限を追加する必要があります。まずこのロールを取得し、それらの権限を追加しましょう。

In [ ]:
import json
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response['roleArn']

policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "GetResourceAPIKey",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceApiKey"
            ],
            "Resource": "*"
        },
        {
            "Sid": "SecretManager",
            "Effect": "Allow",
            "Action": [
                "secretsmanager:GetSecretValue"
            ],
            "Resource": "arn:aws:secretsmanager:*:*:secret:bedrock-agentcore*"
        }
    ]
}
iam_client = boto3.client(
    'iam',
    region_name=region
)

response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

### AgentCore Runtime ステータスの確認
AgentCore Runtime をデプロイしたので、デプロイメントステータスを確認しましょう

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtime の呼び出し

最後に、ペイロードで AgentCore Runtime を呼び出すことができます

<div style="text-align:left">
    <img src="images/invoke.png" width="75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "こんにちは"}, user_id="userid_1234567890")
invoke_response

### 呼び出し結果の処理

呼び出し結果を処理してアプリケーションに含めることができます

In [ ]:
from IPython.display import Markdown, display
response_text = invoke_response['response'][0]
display(Markdown(response_text))

### boto3 で AgentCore Runtime を呼び出す

AgentCore Runtime が作成されたので、任意の AWS SDK で呼び出すことができます。例えば、boto3 の `invoke_agent_runtime` メソッドを使用できます。

In [ ]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    runtimeUserId="userid_1234567890",
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "2×2はいくつですか？"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"EventStream 読み取りエラー: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=runtime_role.split("/")[1],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=runtime_role.split("/")[1],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=runtime_role.split("/")[1]
)

# おめでとうございます！